In [ ]:
# use this notebook to test the data augmentation functions, and to generate ECG photograph datasets (see bottommost cell)

In [ ]:
# imports

from data_augmentation import *
from IPython.core.display_functions import display
import math
import os

In [ ]:
# tips for finding sensible default parameters for the parameter distributions of get_random_ecg_photo:
#
# fix all sigmas to 0 except the parameter(s) to vary; downscale image for faster generation time
# go to desmos.com, input Gaussian/Beta PDF formula with parameters to see how likely a given parameter value will be
# for shadows, we can have either a bright shadow (camera flash) or a dark shadow (normal shadow); we use a U-shaped beta distribution to simulate this

In [ ]:
bg_img = get_background_image(3)  # useful for fixing a specific background image

In [ ]:
# experiment with different parameters here

img, label = get_random_ecg_photo(original_image_scaling_factor_mu=1.0,
                                  original_image_scaling_factor_sd=0.0,
                                  blur_factor_mu=0.1,
                                  blur_factor_sd=0.05,
                                  ecg_paper_scale_mu=1.02,
                                  ecg_paper_scale_sd=0.02,
                                  ecg_paper_y_skew_mu=1.0,
                                  ecg_paper_y_skew_sd=0.08,
                                  rotation_angle_mu=0.0,
                                  rotation_angle_sd=5.0,
                                  ecg_paper_relative_translation_x_mu=0.0,
                                  ecg_paper_relative_translation_x_sd=0.05,
                                  ecg_paper_relative_translation_y_mu=0.0,
                                  ecg_paper_relative_translation_y_sd=0.05,
                                  shadow_color_beta_1=0.3,
                                  shadow_color_beta_2=0.3,
                                  shadow_alpha_beta_1=8.0,
                                  shadow_alpha_beta_2=25.0,
                                  shadow_relative_start_point_mu=0.0,
                                  shadow_relative_start_point_sd=0.1,
                                  shadow_relative_end_point_mu=1.0,
                                  shadow_relative_end_point_sd=0.1,
                                  shadow_blur_factor_mu=50.0,
                                  shadow_blur_factor_sigma=3.0)
                                  #background_image=bg_img)
display(img)

In [ ]:
# dataset generation

import datetime

save_dir = os.path.join('datasets', 'ptb_v')
os.makedirs(save_dir, exist_ok=True)

num_samples_to_generate = 10

for sample_idx in range(num_samples_to_generate):
    photo, data = get_random_ecg_photo()
    scp_codes = data['scp_codes']
    scp_code_str = '_'.join([f'{k}={v}' for k, v in scp_codes.items()])
    rec_date = datetime.datetime.strptime(data['recording_date'], '%Y-%m-%d %H:%M:%S')
    rec_timestamp = (rec_date - datetime.datetime(1970, 1, 1)).total_seconds()
    id_str = str(int(data['patient_id'])) + '_' + str(int(rec_timestamp))
    if len(scp_code_str) > 0:
        scp_code_str = '_' + scp_code_str
    filename = ('%0' + str(math.ceil(math.log10(num_samples_to_generate))) + 'd_' + id_str + scp_code_str + '.jpg') % sample_idx
    photo.convert('RGB').save(os.path.join(save_dir, filename))